# Penguin Analysis with jupyter_cc

This notebook demonstrates using `%cc` magic to work with Claude Code inside Jupyter.

In [ ]:
%load_ext jupyter_cc_magic

## Load Data

Ask Claude to import and explore the penguin dataset.

In [ ]:
%cc Import the penguins dataset from seaborn. Show the first few rows and basic info. No comments in the code.

## Visualize

Create a visualization using the data.

In [ ]:
%cc Create a scatter plot of bill_length_mm vs bill_depth_mm, colored by species. Use matplotlib.

## Summary Statistics

Start a fresh conversation to calculate summary stats.

In [ ]:
%cc_new Calculate summary statistics for body_mass_g grouped by species and sex. Use pandas.

## In-place Editing

Use `%cc_cur` to replace the prompt cell with generated code.

In [ ]:
%cc_cur Create a function that takes a species name and returns a dict of mean measurements